In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 코드 실행 후 세션 다시 시작
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# **library**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder,RobustScaler

from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.svm import SVR

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.inspection import PartialDependenceDisplay

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] =False

import warnings
warnings.filterwarnings('ignore')

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn import set_config

from tqdm import tqdm
import os

In [ ]:
base_path = '/content/drive/MyDrive/24-1 데마 플젝/DATA/'
file_name = 'df+12839+65+log_transform+no_parking_jipgak_n'
SEED = 1886

# **Data import**

In [ ]:
df = pd.read_csv(f"{base_path}{file_name}.csv")
df = df.drop(columns=['date'])
df['month'] = df['month'].astype(object)
df['weekday'] = df['weekday'].astype(object)
df.head()

In [ ]:
df.info()

In [ ]:
# y인 people log변환 여부
is_y_log = False

In [ ]:
X = df.drop(columns=['people'])
y = np.log1p(df['people']) if is_y_log else df['people']

# **Train Test Split**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

# **Model Pipeline**
- numeric : robustscaler
  - 이상치에 민감하게 반응하지 않게 하기 위해 선택
  - tree based model의 경우 스케일링 진행하지 않음
- categorical : onehotencoding
- 모델 학습
  - linear regression
  - elasticnet regression
  - randomforest
  - xgboost
  - lightgbm
  - svr
- 하이퍼파라미터 튜닝 : GridSearchCV
- 모델 평가

## **Functions**

In [ ]:
# 특정 모델만 pipeline에 넣어서 학습 후 학습된 모델 반환해주는 함수
def get_model_pipeline(model_name, CV=5, pipeline=False):
    for name, model, params in models_with_scaling + models_without_scaling:
        if name == model_name:
            if model_name in [m[0] for m in models_with_scaling]:
                pl = Pipeline(steps=[('preprocessor', preprocessor_with_scaling), ('model', model)])
            else:
                pl = Pipeline(steps=[('preprocessor', preprocessor_without_scaling), ('model', model)])

            if pipeline: return pl

            grid_search = GridSearchCV(pl, param_grid=params, scoring='neg_mean_squared_error', cv=CV)
            grid_search.fit(X_train, y_train)

            print(grid_search.best_params_)
            return grid_search.best_estimator_
    return None

In [ ]:
# 모델 평가지표(RMSE, r-squared) 뽑기
def evaluate_model(model_pipeline, y_test, is_y_log = False):
    y_pred = model_pipeline.predict(X_test)
    if is_y_log:
      y_pred = np.expm1(y_pred)  # 역변환
      y_test = np.expm1(y_test)  # 역변환

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    print(f'RMSE: {rmse}')
    print(f'R2 Score: {r2}')

    return y_pred, rmse, r2

In [ ]:
def plot_feature_importance(model_pipeline, model_name = None, rmse = None, file_name = None, is_save=False, feature_names_kor = None):
    if hasattr(model_pipeline.named_steps['model'], 'feature_importances_'):
        feature_importances = model_pipeline.named_steps['model'].feature_importances_
        #feature_importances = model_pipeline.named_steps['model'].booster_.feature_importance(importance_type='gain') # for lightgbm
        feature_names = numeric_features +
          list(model_pipeline.named_steps['preprocessor'].transformers_[1][1].get_feature_names_out(categorical_features))
        plt.figure(figsize=(8, 8))
        ftr_importances = pd.Series(feature_importances, index=feature_names)
        ftr_top = ftr_importances.sort_values(ascending=False)[:20]

        if feature_names_kor is None:
          feature_names_kor = {feature: feature for feature in ftr_top.index}

        # 변수명을 한글로 변경
        top_features_kor = [feature_names_kor[feature] for feature in ftr_top.index]


        sns.barplot(x=ftr_top, y=top_features_kor, color='lightseagreen')
        plt.title(f"{model_name} {rmse} Feature Importance")
        plt.xlabel(None)
        plt.ylabel(None)
        plt.tight_layout()
        if is_save:
          plt.savefig(f'/content/drive/MyDrive/24-1 데마 플젝/Feature Importance Plots/{model_name} {rmse} {file_name}.png')
        plt.show()
        return ftr_top
    else:
        print('Feature importances are not available for this model.')

In [ ]:
def plot_pdp(model_pipeline, features, feature_names_kor):
    # Partial Dependence Plot을 생성하는 함수
    fig, ax = plt.subplots(2, 5, figsize=(20, 8), sharey=True)
    feature_names = numeric_features +
      list(model_pipeline.named_steps['preprocessor'].transformers_[1][1].get_feature_names_out(categorical_features))
    X_transformed = model_pipeline.named_steps['preprocessor'].transform(X_train)
    plt.rc('axes', labelsize=12)
    PartialDependenceDisplay.from_estimator(model_pipeline.named_steps['model'],
                                            pd.DataFrame(X_transformed, columns=feature_names),
                                            features, line_kw={"color": "lightseagreen"}, grid_resolution=50, ax=ax)

    for idx, axi in enumerate(ax.flatten()):
        if idx < len(features):
            feature_index = features[idx]
            axi.set_xlabel(feature_names_kor[feature_index], fontsize=12)

    plt.tight_layout()
    plt.show()

In [ ]:
def plot_observed_vs_predicted(y_test, y_pred, model_name = None, is_y_log = False):
    # Observed vs Predicted 산점도를 생성하는 함수
    plt.figure(figsize=(6, 6))
    if is_y_log: y_test = np.expm1(y_test)
    plt.scatter(y_test, y_pred, label='Testing data', color='lightseagreen', alpha=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='black', linestyle='--')
    plt.xlabel('Observed')
    plt.ylabel('Predicted')
    plt.title(f'Observed vs Predicted ({model_name})')
    plt.xlim([min(y_test)-1000, max(y_test)+1000])
    plt.ylim([min(y_test)-1000, max(y_test)+1000])
    plt.gca().set_aspect('equal', adjustable='box')
    plt.show()

## **Scaling & OnehotEncoding**

In [ ]:
# 컬럼 분리
numeric_features = X.select_dtypes(include=['float64', 'int']).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

In [ ]:
categorical_features

In [ ]:
# 비-tree 기반 모델을 위한 전처리 파이프라인 (수치형 변수 스케일링 포함)
preprocessor_with_scaling = ColumnTransformer(
    transformers=[
        ('num', RobustScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# tree 기반 모델을 위한 전처리 파이프라인 (수치형 변수 스케일링 생략)
preprocessor_without_scaling = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

## **모델 및 하이퍼파라미터 정의**

In [ ]:
# 모델 리스트
models_with_scaling = [
    ('Linear Regression', LinearRegression(), {}),
    ('ElasticNet', ElasticNet(), {'model__alpha': [0.3], 'model__l1_ratio': [1.0]}),
]

models_without_scaling = [
    ('RandomForest', RandomForestRegressor(random_state=SEED), {'model__n_estimators': [400], 'model__max_depth': [20],
                                                                'model__min_samples_leaf' : [4], 'model__min_samples_split' : [3]}),
    ('XGBRegressor', XGBRegressor(random_state=SEED), {'model__n_estimators': [270], 'model__learning_rate': [0.06],
                                                       'model__max_depth' : [5], 'model__min_child_weight' : [4], 'model__gamma' : [0],
                                                       'model__subsample' : [0.8], 'model__colsample_bytree' : [0.5],
                                                       'model__reg_alpha': [2], 'model__reg_lambda': [0.0001]}),
    ('LGBMRegressor', LGBMRegressor(random_state=SEED, verbose=-1), {'model__n_estimators': [270], 'model__learning_rate': [0.06],
                                                                     'model__num_leaves' : [31], 'model__min_data_in_leaf' : [20],
                                                                     'model__max_depth' : [-1]})
]

## **모든 모델 학습 및 RMSE 비교**

In [ ]:
# 5가지 모델에 대해 평가 및 RMSE 저장
CV = 5
save_models = {}
rmses = {}
for name, model, params in tqdm(models_with_scaling + models_without_scaling):
    if name in [m[0] for m in models_with_scaling]:
        pl = Pipeline(steps=[('preprocessor', preprocessor_with_scaling), ('model', model)])
    else:
        pl = Pipeline(steps=[('preprocessor', preprocessor_without_scaling), ('model', model)])

    grid_search = GridSearchCV(pl, param_grid=params, scoring='neg_mean_squared_error', cv=CV)
    grid_search.fit(X_train, y_train)
    model_pipeline = grid_search.best_estimator_

    save_models[name] = model_pipeline
    print(f'\n******Evaluating {name}******')
    y_pred, rmse, r2 = evaluate_model(model_pipeline, y_test, is_y_log)
    rmses[name] = rmse

In [ ]:
# RMSE를 바탕으로 bar plot 생성
plt.figure(figsize=(10, 6))
plt.barh([*rmses], rmses.values(), color='lightseagreen')
plt.xlabel('RMSE')
#plt.ylabel('RMSE')
plt.title('Model Comparison based on RMSE')
#plt.xticks(rotation=45)
plt.rc('font', size=12)
plt.show()

In [ ]:
min_model = min(rmses, key=rmses.get)
min_rmse = rmses[min_model]
best_model = save_models[min_model]

In [ ]:
best_model

## **특정 모델 학습 및 평가**
- model_name
  - 'RandomForest'
  - 'XGBRegressor'
  - 'LGBMRegressor'

### **RandomForest**

In [ ]:
# 특정 모델 입력 후 학습
model_name = 'RandomForest'
model_pipeline = get_model_pipeline(model_name, CV=5) # default CV = 5

In [ ]:
model_pipeline = save_models[model_name]

In [ ]:
# 특정 모델 평가
if model_pipeline:
    y_pred, rmse, r2 = evaluate_model(model_pipeline, y_test, is_y_log)
else:
    print(f'Model {model_name} not found in the list of models.')

In [ ]:
feature_names_kor = {
    'delivery_zone': '배달존 수',
    'sub_walk_min': '역까지의 최소 도보 거리',
    'cafe_n': '카페 수',
    'green_area': '녹지 면적',
    'drinking_fountain': '음수대 수',
    'access': '공원 접근 시설 수',
    'avg_temp': '평균 기온',
    'hangang_뚝섬': '뚝섬',
    'orak_n': '오락시설 수',
    'restaurant_n': '식당 수',
    'humidity': '습도',
    'covid_people': '코로나 확진자 수',
    'rain': '강수량',
    'temp_diff': '일교차',
    'weekend': '주말여부',
    'log_cafe_revenue': 'log(카페 매출 총액)',
    'log_food_cost': 'log(음식 지출 총액)',
    'covid_step': '사회적 거리두기 단계',
    'log_drink_cost': 'log(유흥 지출 총액)',
    'log_leisure_cost': 'log(여가 문화 지출 총액)'
}

In [ ]:
# Feature importance plot
ftr_top = plot_feature_importance(model_pipeline, model_name = model_name, rmse = "",
                                  file_name = file_name, is_save=False, feature_names_kor = feature_names_kor)

In [ ]:
# Partial Dependence Plot (feature importance 기준 상위 10개만)
plot_pdp(model_pipeline, ftr_top.index[:10], feature_names_kor)

In [ ]:
plot_observed_vs_predicted(y_test, y_pred, model_name = model_name)

### **XGBRegressor**

In [ ]:
# 특정 모델 입력 후 학습
model_name = 'XGBRegressor'
model_pipeline = get_model_pipeline(model_name, CV=5) # default CV = 5

In [ ]:
# 특정 모델 평가
if model_pipeline:
    y_pred, rmse, r2 = evaluate_model(model_pipeline, y_test, is_y_log)
else:
    print(f'Model {model_name} not found in the list of models.')

In [ ]:
# Featrue importance plot
ftr_top = plot_feature_importance(save_models[model_name], model_name = model_name,
                                  rmse = str(round(rmses[model_name], 3)), file_name = file_name, is_save=True)

### **LGBMRegressor**

In [ ]:
# 특정 모델 입력 후 학습
model_name = 'LGBMRegressor'
model_pipeline = get_model_pipeline(model_name, CV=5) # default CV = 5

In [ ]:
model_pipeline = save_models[model_name]

In [ ]:
# 특정 모델 평가
if model_pipeline:
    y_pred, rmse, r2 = evaluate_model(model_pipeline, y_test, is_y_log)
else:
    print(f'Model {model_name} not found in the list of models.')

In [ ]:
feature_names_kor = {
    'avg_temp': '평균 기온',
    'avg_wind': '평균 풍속',
    'cafe_n': '카페 수',
    'restaurant_n': '식당 수',
    'log_orak_revenue': 'log(오락 지출 총액)',
    'log_convenience_store_revenue': 'log(편의점 매출 총액)',
    'convenience_store_n': '편의점 수',
    'log_restaurant_revenue': 'log(식당 매출 총액)',
    'orak_n': '오락시설 수',
    'culture': '문화 시설 수',
    'humidity': '습도',
    'covid_people': '코로나 확진자 수',
    'rain': '강수량',
    'temp_diff': '일교차',
    'weekend': '주말여부',
    'log_cafe_revenue': 'log(카페 매출 총액)',
    'log_food_cost': 'log(음식 지출 총액)',
    'covid_step': '사회적 거리두기 단계',
    'log_drink_cost': 'log(유흥 지출 총액)',
    'log_leisure_cost': 'log(여가 문화 지출 총액)'
}

In [ ]:
model_pipeline.named_steps['model'].booster_.feature_importance(importance_type='gain')

In [ ]:
# Featrue importance plot
ftr_top = plot_feature_importance(model_pipeline, model_name = model_name, rmse = "",
                                  file_name = file_name, is_save=False, feature_names_kor = None)